In [1]:
import os
import yfinance as yf
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis, skewnorm

def analyze_stock_data(tickers, start_date, end_date, trading_days, risk_free_rate):
    # Download S&P 500 data
    sp500 = yf.download('^GSPC', start=start_date, end=end_date)
    sp500['Daily Return'] = sp500['Close'].pct_change().dropna()

    # Initialize a list to store metrics
    metrics = []

    for company, ticker in tickers.items():
        try:
            # Download stock data
            stock_data = yf.download(ticker, start=start_date, end=end_date)
            if stock_data.empty:
                raise ValueError(f"No data for {company} ({ticker})")

            # Calculate daily return
            stock_data['Daily Return'] = stock_data['Close'].pct_change().dropna()

            # Calculate mean return, volatility, and Sharpe ratio
            mean_return = stock_data['Daily Return'].mean() * trading_days
            volatility = stock_data['Daily Return'].std(ddof=1) * np.sqrt(trading_days)
            sharpe_ratio = (mean_return - risk_free_rate) / volatility
            beta = stock_data['Daily Return'].cov(sp500['Daily Return']) / sp500['Daily Return'].var(ddof=1)

            # Calculate skewness and kurtosis (handle NaN issues by ensuring enough data)
            daily_returns_valid = stock_data['Daily Return'].dropna()
            skewness = skew(daily_returns_valid)
            kurt = kurtosis(daily_returns_valid)

            # 1. Empirical probability of negative returns
            prob_negative_empirical = (daily_returns_valid < 0).mean()
            mu = daily_returns_valid.mean()
            sigma = daily_returns_valid.std()
            prob_negative_skewnorm = skewnorm.cdf(0, skewness, loc=mu, scale=sigma)

            # Append calculated metrics
            metrics.append([
                company, 
                mean_return, 
                volatility, 
                sharpe_ratio, 
                skewness, 
                kurt, 
                beta, 
                prob_negative_empirical, 
                prob_negative_skewnorm
            ])

        except Exception as e:
            print(f"Error with {company} ({ticker}): {e}")
            continue

    # Convert metrics list to a DataFrame
    columns = [
        'Company', 'Mean Return', 'Volatility', 'Sharpe Ratio', 'Skewness', 
        'Kurtosis', 'Beta', 'Prob Negative (Empirical)', 'Prob Negative (Skewnorm)'
    ]
    data_frame = pd.DataFrame(metrics, columns=columns)

    
    data_frame.sort_values(by='Sharpe Ratio', ascending=False, inplace=True)        # Sort by 'Sharpe Ratio'
    os.makedirs("Excel", exist_ok=True)                                             # Create Excel folder if it doesn't exist
    file_path = os.path.join("Excel", f'rf {risk_free_rate} days {trading_days} from {start_date} to {end_date}.xlsx')    
    data_frame.to_excel(file_path, index=False)                                     # Save to Excel

    return data_frame

In [3]:
# Read stock from file
with open('stocks.txt', 'r') as file:
    tickers = {line.strip(): line.strip() for line in file}

start_date = '2014-04-10'
end_date = '2024-10-10'
risk_free_rate = 0.00
days = 2

analyze_stock_data(tickers, start_date, end_date, days, risk_free_rate)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

,Company,Mean Return,Volatility,Sharpe Ratio,Skewness,Kurtosis,Beta,Prob Negative (Empirical),Prob Negative (Skewnorm)
107,NVDA,0.005181,0.042385,0.122245,0.652871,8.754023,1.723477,0.453066,0.282225
92,MELIN.MX,0.006391,0.055438,0.115285,1.000943,7.195260,1.117370,0.451113,0.218421
84,LLY,0.002374,0.024178,0.098171,1.032855,11.785675,0.695208,0.465556,0.217975
20,AVGO,0.003122,0.032216,0.096919,-0.030082,7.400556,1.377216,0.464421,0.482231
38,COST,0.001765,0.018940,0.093191,-0.300131,8.848555,0.725731,0.454958,0.566335
...,...,...,...,...,...,...,...,...,...
134,SPCE,-0.001252,0.074189,-0.016879,0.869583,6.188651,1.845091,0.496321,0.276950
140,TLEVISACPO.MX,-0.001102,0.033161,-0.033245,0.823549,8.300616,0.781116,0.515714,0.290155
132,SKLZ,-0.003260,0.080864,-0.040315,0.960091,3.912505,2.254555,0.529884,0.267975
127,RIVN,-0.003463,0.073832,-0.046903,0.131783,2.866720,2.271765,0.532148,0.471544
